In [ ]:
"""
what we've to do is:

if(the encoded word has two parts):
    -we cant really split it like a normal english word with suffix-mid-prefix, so we've to try to expand the begining, middle and ending properly
    -split this and then encode the word now, without splitting it further.
    -after getting the op of the transformer, try to fine tune the model using data that can be used to generate words that follow punarchi vidhi
(ALWAYS CLEAR THE OUTPUT CELSS BEFORE PUSHING)
"""

In [ ]:
#imports
import numpy as np
import re, collections

In [ ]:
#code for reading the file
"""some wget thingy"""
!wget https://raw.githubusercontent.com/Ishikahooda/Tamil-English-Dataset/master/Dataset/data.ta1

In [ ]:
# read it in to inspect it
with open('data.ta1', 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:100])

In [ ]:
#function that returns the mapping of words with their resp freqs
def getStats(vocab):
    #retunrs an empty dict with integer vals
    pairs=collections.defaultdict(int)
    #for loop that iterates through each word and in in the vocab dict
    for word, freq in vocab.items():
        #splits the words using an empty space character
        symbols=word.split()
        #iterates through the split words
        for i in range(len(symbols)-1):
            #updates the frequencies of occurrence of the current char with the next char
            pairs[symbols[i], symbols[i+1]]+=freq
    #the pairs after BPE is returned
    return pairs

#This returns a new vocab using the given pairs and vocab, where the pairs of words are merged together wherever they appear in the vocab 
def mergeVocab(pair, vocabIn):
    #an empty dictionary that would contain the bit pairs after BPE
    vocabOut={}
    #the words are joined with the hidden escape character for each word
    bigram=re.escape(' '.join(pair))
    #the bigram regular exp is converted into a pattern obj
    p=re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    #the words in the input vocab is iterated
    for wordIn in vocabIn:
        #replaces the occurrence of the given pair in p with the iterated word
        wordOut=p.sub(''.join(pair), wordIn)
        #for the key 'wordOut' in the output vocab dict, we have the value of the iterated word from the input vocab dict
        vocabOut[wordOut]=vocabIn[wordIn]
    #the output vocab dictionary is returned
    return vocabOut

#returns the vocan from the given document or string of words
def getVocab(data):
    #creates and empty dict with integer values
    vocab=collections.defaultdict(int)
    #iterating through each line of the document
    for line in data:
        #iterating through each word in the line
        for word in line.split():
            #the dict would be like -> ex: {'hello</w>':4, ...}
            vocab[' '.join(list(word))+' <w>']+=1
    #returns the vocab
    return vocab

#returns byte pair encoded vocab with the given corpus and max merged pairs
def bytePairEnc(data, n):
    #this returns the vocab from the given corpus
    vocab=getVocab(data)
    #this loop iterates for n timess
    for _ in range(n):
        #function that returns the mapping of words with their resp freqs with the given vocab
        pairs=getStats(vocab)
        #this returns the best pairs from the given vocab
        topPairs=max(pairs, key=pairs.get)
        #This returns a new vocab using the given pairs and vocab, where the pairs of words are merged together wherever they appear in the vocab 
        mergedVocab=mergeVocab(topPairs, vocab)
    #returns the byte pair encoded vocab
    return mergedVocab

In [ ]:
#getting the corpus and the number of merges
corpus=text
data=corpus.split('.')
n=200
bpePairs=bytePairEnc(data, n)
print(bpePairs)

In [ ]:
#have a dict with list vals for each word that represents word=word1+word2
tamWords={"மூதூர்":["முதுமை", "ஊர்"], "பாசி":["பசுமை", "இ"], "பைங்கொடி":["பசுமை", "கொடி"]}

#try having a dict with completely different words that follow this trend. Try learning the trends in this to apply it in for new data